In [2]:
𝛕 = [];      # split times
𝛏 = [];      # split values
𝛅 = [];      # split dimensions
Leaves = []; 

In [3]:
function Sample_Mondrian_Tree!(λ::Float64,
                               X::Array{Float64,2},
                               Y::Array{Int64})
    𝛕 = [];                # split times
    𝛏 = [];                # split values
    𝛅 = [];                # split dimensions
    Leaves = [];
    Indices = [];          # data points in node
    Tree = [𝛕,𝛏,𝛅,Leaves, Indices];
    Sample_Mondrian_Block!(Tree, λ, X)
end

function Sample_Mondrian_Block!(Tree::Array{Any},
                                λ::Float64,
                                X::Array{Float64,2})
    push!(Indices, collect(1:size(X,1)))
    push
    
    # sample the time
    E = rand(Exponential(1/Linear_dimension(Θ)))
    if j.node_type[3]==true
        τₚ = 0
    else
        τₚ = (get(j.parent)).τ
    end
    # if split occured in time
    if τₚ + E < λ
        # get split dimension and cut position
        # A2 -> lines 6,7
        d,x = sample_split_dimension(Θ)
        # update node j's data
        j.δ = d
        j.ζ = x
        j.τ = τₚ+E
        Θᴸ = copy(Θ)
        # look at this copy
        Θᴿ = copy(Θ)
        # Left and Right children have constricted boxes
        Θᴸ.Intervals[d,2]=x
        Θᴿ.Intervals[d,1]=x
        # check there is actually data here
        Xᴿ = get_data_indices(Θᴿ,X,d)
        Xᴸ = get_data_indices(Θᴸ,X,d)
        # strictly binary tree
        if (size(Xᴿ,1)>0 && size(Xᴸ,1)>0)
            right = Mondrian_Node(0.0, [true,false,false])
            right.parent = j
            # data changes A2 -> lines 8,9,10
            right.Θ = Θᴿ
            get_count(right, Y[Xᴿ], length(j.c))
            right.Gₚ=zeros(size(j.c,1))
            j.right = right

            left = Mondrian_Node(0.0, [true,false,false])
            left.parent = j
            left.Θ = Θᴸ
            get_count(left, Y[Xᴸ], length(j.c))
            left.Gₚ = zeros(size(j.c,1))
            j.left = left

            # recurse
            Sample_Mondrian_Block!(left, get(left.Θ), λ, Tree, X[Xᴸ,:], Y[Xᴸ])
            Sample_Mondrian_Block!(right,get(right.Θ),λ, Tree, X[Xᴿ,:], Y[Xᴿ])
        # set j as a leaf for no data/ not binary
        else
            j.τ = λ
            j.node_type = [false,true,false]
            push!(Tree.leaves,j)
            return
        end
    # set j as leaf for time out
    else
        j.τ = λ
        j.node_type = [false,true,false]
        push!(Tree.leaves,j)
        return
    end
end

LoadError: [91mUndefVarError: Mondrian_Node not defined[39m